In [1]:

#r "nuget: Microsoft.SemanticKernel, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Core, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230629.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.17.230629.1-preview Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.17.230629.1-preview Microsoft.SemanticKernel.Core, 0.17.230629.1-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;

IKernel semanticKernel = Kernel.Builder
                    .WithOpenAITextCompletionService(
                        "text-davinci-003",                     // OpenAI Model Name
                        "sk-<KEY-HERE>" // OpenAI Key
                    )
                    .Build();

# Skill 1: Register Native Function

I am copying directily from https://github.com/microsoft/semantic-kernel two basic native function

In [59]:
// Copyright (c) Microsoft. All rights reserved.

using System.ComponentModel;
using Microsoft.SemanticKernel.SkillDefinition;

public sealed class StaticTextSkill
{
    [SKFunction, Description("Change all string chars to uppercase")]
    public static string Uppercase([Description("Text to uppercase")] string input) =>
        input.ToUpperInvariant();

    [SKFunction, Description("Append the day variable")]
    public static string AppendDay(
        [Description("Text to append to")] string input,
        [Description("Value of the day to append")] string day) =>
        input + day;

    [SKFunction, Description("Get today's day")]
    public static string GetTodayDay() =>
        DateTime.Today.DayOfWeek.ToString();
}

// register native function
semanticKernel.ImportSkill(new StaticTextSkill(), "StaticTextSkill");

# Define PLanner

In [60]:
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Stepwise;

var plannerConfig = new Microsoft.SemanticKernel.Planning.Stepwise.StepwisePlannerConfig();
plannerConfig.ExcludedFunctions.Add("TranslateMathProblem");
plannerConfig.MinIterationTimeMs = 1500;
plannerConfig.MaxTokens = 1000;

StepwisePlanner planner = new StepwisePlanner(semanticKernel, plannerConfig);


In [70]:
var plan = planner.CreatePlan("convert this text to upper case and append today's day : Hi, today is lovely day :  ");

In [71]:
var result = await plan.InvokeAsync(semanticKernel.CreateNewContext());


In [72]:
if (result.Variables.TryGetValue("stepCount", out string? stepCount))
{
        Console.WriteLine("Steps Taken: " + stepCount);
}

if (result.Variables.TryGetValue("agentScratchPad", out string? agentThinking))
{
        Console.WriteLine(" " + agentThinking);
}

Console.WriteLine("\n\nFinal Answer is : " + result);

Steps Taken: 5
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
[THOUGHT] 
[OBSERVATION] System step error, no thought or action found. Please give a valid thought and/or action.
[THOUGHT] I need to use the StaticTextSkill.Uppercase and StaticTextSkill.AppendDay functions to convert the text to uppercase and append today's day.
[ACTION] {"action": "StaticTextSkill.Uppercase","action_variables": {"input":"Hi, today is lovely day"}}
[OBSERVATION] HI, TODAY IS LOVELY DAY
[THOUGHT] [ACTION] {"action": "StaticTextSkill.GetTodayDay"}
[ACTION] {"action": "StaticTextSkill.GetTodayDay","action_variables": null}
[OBSERVATION] Saturday
[THOUGHT] I need to use the StaticTextSkill.AppendDay function to append the day to the text.
[ACTION] {"action": "StaticTextSkill.AppendDay","action_variables": {"input":"HI, TODAY IS LOVELY DAY","day":"Saturday"}}
[OBSERVATION] HI, TODAY IS LOVELY DAYSaturday
[THOUGHT]


Final Answer is : The text "Hi, tod